# Driver notebook

This is an auto-generated notebook created by an AI Playground export. We generated three notebooks in the same folder:
- [agent]($./agent): contains the code to build the agent.
- [config.yml]($./config.yml): contains the configurations.
- [**driver**]($./driver): logs, evaluate, registers, and deploys the agent.

This notebook uses Mosaic AI Agent Framework ([AWS](https://docs.databricks.com/en/generative-ai/retrieval-augmented-generation.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/retrieval-augmented-generation)) to deploy the agent defined in the [agent]($./agent) notebook. The notebook does the following:
1. Logs the agent to MLflow
2. Evaluate the agent with Agent Evaluation
3. Registers the agent to Unity Catalog
4. Deploys the agent to a Model Serving endpoint

## Prerequisities

- Address all `TODO`s in this notebook.
- Review the contents of [config.yml]($./config.yml) as it defines the tools available to your agent, the LLM endpoint, and the agent prompt.
- Review and run the [agent]($./agent) notebook in this folder to view the agent's code, iterate on the code, and test outputs.

## Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See docs ([AWS](https://docs.databricks.com/en/generative-ai/deploy-agent.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/deploy-agent)) for details

In [0]:
%pip install -U -qqqq databricks-agents mlflow langchain==0.2.16 langchain_core langchain-community==0.2.16 langgraph==0.2.16 pydantic
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.5.1 requires pydantic<2,>=1.8.1, but you have pydantic 2.9.2 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


### Log the `agent` as an MLflow model
Log the agent as code from the [agent]($./agent) notebook. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

In [0]:
# Log the model to MLflow
import os
import mlflow

input_example = {
    "messages": [
        {
            "role": "user",
            "content": "I am reviewing my investment portfolio. What should I do with ICICI stock?"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.langchain.log_model(
        lc_model=os.path.join(
            os.getcwd(),
            'agent',
        ),
        pip_requirements=[
            f"langchain==0.2.16",
            f"langchain-community==0.2.16",
            f"langgraph==0.2.16",
            f"pydantic",
        ],
        model_config="config.yml",
        artifact_path='agent',
        input_example=input_example,
    )

2024/09/29 19:16:43 WARNING mlflow.models.utils: The model file uses magic commands which have been commented out. To ensure your code functions correctly, make sure that it does not rely on these magic commands for correctness.


<uc_function_call>{
  "id": "call_5ba8c119-4a74-440f-8562-1709d6d2a137",
  "name": "sarbanimaiti_catalog__agent_demo__finance_stock_quote",
  "arguments": "{\"query\": \"Hindalco stock price\"}"
}</uc_function_call>

 ------------------------------------------------------------

<uc_function_result>{
  "id": "call_5ba8c119-4a74-440f-8562-1709d6d2a137",
  "content": "{\"format\": \"SCALAR\", \"value\": \"{'quoteResponse': {'result': [], 'error': None}}\", \"truncated\": false}"
}</uc_function_result>

 ------------------------------------------------------------

<uc_function_call>{
  "id": "call_5b83a793-e681-4e0e-bde7-7b097210031a",
  "name": "sarbanimaiti_catalog__agent_demo__finance_insight_api",
  "arguments": "{\"query\": \"Hindalco stock insight\"}"
}</uc_function_call>

 ------------------------------------------------------------

<uc_function_result>{
  "id": "call_5b83a793-e681-4e0e-bde7-7b097210031a",
  "content": "{\"format\": \"SCALAR\", \"value\": \"[]\", \"truncated\": f

/local_disk0/.ephemeral_nfs/envs/pythonEnv-3dfc6aa7-8147-49e9-a82d-818bc838289e/lib/python3.11/site-packages/mlflow/models/utils.py:523: FutureWarning: Since MLflow 2.16.0, we no longer convert dictionary input example to pandas Dataframe, and directly save it as a json object. If the model expects a pandas DataFrame input instead, please pass the pandas DataFrame as input example directly.
  example = _Example(input_example)


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

To provide a recommendation on whether Hindalco shares are a good choice to invest in, I need to first identify the company's stock ticker symbol. The stock ticker symbol for Hindalco Industries Limited is HINDALCO.

Next, I will use the finance_stock_quote tool to get the current stock price of Hindalco.

<function=sarbanimaiti_catalog__agent_demo__finance_stock_quote>{"query": "HINDALCO"}</function>

Please provide the output of the above function call, and I will proceed with the analysis.

Once I have the current stock price, I will use the finance_insight_api tool to get a summary of the stock's performance.

<function=sarbanimaiti_catalog__agent_demo__finance_insight_api>{"query": "HINDALCO"}</function>

Please provide the output of the above function call, and I will provide a final recommendation on whether to buy, hold, or sell Hindalco shares.

 ------------------------------------------------------------



2024/09/29 19:17:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run upbeat-gnu-756 at: https://e2-demo-field-eng.cloud.databricks.com/ml/experiments/4320825364211507/runs/1fbf4fe7fe314bf489b8ea2f8fee1d6a.
2024/09/29 19:17:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://e2-demo-field-eng.cloud.databricks.com/ml/experiments/4320825364211507.


## Evaluate the agent with [Agent Evaluation](https://docs.databricks.com/generative-ai/agent-evaluation/index.html)

You can edit the requests or expected responses in your evaluation dataset and run evaluation as you iterate your agent, leveraging mlflow to track the computed quality metrics.

In [0]:
import pandas as pd

eval_examples = [
    {
        "request":  "I am reviewing my investment portfolio. What should I do with ICICI stock?",

        "expected_response": "Hold. The current stock price of ICICI Bank Limited (ICICIBANK.NS) is 1306.5 INR, with a regular market change of -1.6967044%. The 52-week range is 899.0 - 1362.35, and the average analyst rating is \"1.4 - Strong Buy\". However, the forward PE ratio is 34.43595, which is relatively high. Considering the mixed signals, it is recommended to hold the stock for now and monitor its performance before making any further decisions."
    }
    ,
    {
        "request": "What should be the choice for CIPLA?",
      
        "expected_response": "Buy. The current stock price of Cipla Limited (CIPLA.NS) is 1674.5 INR, with a regular market change of 3.2494729%. The 52-week range is 1132.0 - 1683.0, and the average analyst rating is \"2.4 - Buy\". The forward PE ratio is 26.017714, which is relatively low. Considering the positive signals, it is recommended to buy the stock."
    },
    {
        "request": "I have 1000 Reliance stocks. Should I hold them?",
            
        "expected_response": "Hold The current stock price of Reliance Industries Limited (RELIANCE.NS) is 3047.05 INR, with a regular market change of 1.7073383%. The 52-week range is 2220.3 - 3217.6, and the average analyst rating is \"2.1 - Buy\". However, the forward PE ratio is 42.408493, which is relatively high. Considering the mixed signals, it is recommended to hold the stock for now and monitor its performance before making any further decisions."
    }
     
]

eval_dataset = pd.DataFrame(eval_examples)
display(eval_dataset)

request,expected_response
I am reviewing my investment portfolio. What should I do with ICICI stock?,"Hold. The current stock price of ICICI Bank Limited (ICICIBANK.NS) is 1306.5 INR, with a regular market change of -1.6967044%. The 52-week range is 899.0 - 1362.35, and the average analyst rating is ""1.4 - Strong Buy"". However, the forward PE ratio is 34.43595, which is relatively high. Considering the mixed signals, it is recommended to hold the stock for now and monitor its performance before making any further decisions."
What should be the choice for CIPLA?,"Buy. The current stock price of Cipla Limited (CIPLA.NS) is 1674.5 INR, with a regular market change of 3.2494729%. The 52-week range is 1132.0 - 1683.0, and the average analyst rating is ""2.4 - Buy"". The forward PE ratio is 26.017714, which is relatively low. Considering the positive signals, it is recommended to buy the stock."
I have 1000 Reliance stocks. Should I hold them?,"Hold The current stock price of Reliance Industries Limited (RELIANCE.NS) is 3047.05 INR, with a regular market change of 1.7073383%. The 52-week range is 2220.3 - 3217.6, and the average analyst rating is ""2.1 - Buy"". However, the forward PE ratio is 42.408493, which is relatively high. Considering the mixed signals, it is recommended to hold the stock for now and monitor its performance before making any further decisions."


In [0]:
import mlflow
import pandas as pd

with mlflow.start_run(run_id=logged_agent_info.run_id):
    eval_results = mlflow.evaluate(
        f"runs:/{logged_agent_info.run_id}/agent",  # replace `chain` with artifact_path that you used when calling log_model.
        data=eval_dataset,  # Your evaluation dataset
        model_type="databricks-agent",  # Enable Mosaic AI Agent Evaluation
    )

# Review the evaluation results in the MLFLow UI (see console output), or access them in place:
display(eval_results.tables['eval_results'])

To provide a recommendation on whether Hindalco shares are a good choice to invest in, I need to first identify the company's stock ticker symbol. The stock ticker symbol for Hindalco Industries Limited is HINDALCO.

Next, I will use the finance_stock_quote tool to get the current stock price of Hindalco.

<function=sarbanimaiti_catalog__agent_demo__finance_stock_quote>{"query": "HINDALCO"}</function>

Please provide the output of the above function call.

Once I have the current stock price, I will use the finance_insight_api tool to get the financial summary of Hindalco.

<function=sarbanimaiti_catalog__agent_demo__finance_insight_api>{"query": "HINDALCO"}</function>

Please provide the output of the above function call.

Based on the current stock price and financial summary, I will provide a recommendation on whether to buy, hold, or sell Hindalco shares.

 ------------------------------------------------------------



Evaluating:   0%|          | 0/3 [Elapsed: 00:00, Remaining: ?]

<!DOCTYPE html>
 
 
 Evaluation output 
 
 
 
 
 
 
 
 <!-- AI icon -->
 
 
 <path fill="currentColor" fill-rule="evenodd"
 d="M4 9a.75.75 0 0 1 .739.621l.152.877a.75.75 0 0 0 .61.61l.878.153a.75.75 0 0 1 0 1.478l-.877.152a.75.75 0 0 0-.61.61l-.153.878a.75.75 0 0 1-1.478 0l-.152-.877a.75.75 0 0 0-.61-.61l-.878-.153a.75.75 0 0 1 0-1.478l.877-.152a.75.75 0 0 0 .61-.61l.153-.878A.75.75 0 0 1 4 9Zm0 2.92a2.289 2.289 0 0 1-.08.08c.028.026.054.052.08.08.026-.028.052-.054.08-.08a2.289 2.289 0 0 1-.08-.08ZM10 0c.36 0 .67.257.737.611l.264 1.398A3.75 3.75 0 0 0 13.99 5l1.398.264a.75.75 0 0 1 0 1.474l-1.398.264A3.75 3.75 0 0 0 11 9.99l-.264 1.398a.75.75 0 0 1-1.474 0l-.264-1.398A3.75 3.75 0 0 0 6.01 7l-1.398-.264a.75.75 0 0 1 0-1.474l1.398-.264A3.75 3.75 0 0 0 9 2.01L9.263.611A.75.75 0 0 1 10 0Zm0 3.682A5.256 5.256 0 0 1 7.682 6 5.256 5.256 0 0 1 10 8.318 5.256 5.256 0 0 1 12.318 6 5.256 5.256 0 0 1 10 3.682Z"
 clip-rule="evenodd"> 
 
 Evaluation completed: 2/3 pass all assessments 
 
 
 
 
 ⚠️ Skipped metric 'context_sufficiency' due to missing input fields: retrieved_context 
 
 ⚠️ Skipped metric 'groundedness' due to missing input fields: retrieved_context 
 
 
 
 
 
 
 <!-- "Open in a new tab" icon -->
 
 
 
 
 View evaluation results
 
 
 
 <!-- "Open in a new tab" icon -->
 
 
 
 
 Compare evaluations across runs
 
 
 
 
 Python APIs: 
 
 mlflow.evaluate() returns a result object that is an instance of EvaluationResult . 
 
 
 result.metrics : A dictionary of aggregated evaluation results.
 
 
 result.tables['eval_results'] : A DataFrame of per-request evaluation results.
 
 

 
 For more details, see the mlflow.evaluate 
 API reference or the MLflow
 LLM Evaluate Quickstart .

2024/09/29 19:19:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run upbeat-gnu-756 at: https://e2-demo-field-eng.cloud.databricks.com/ml/experiments/4320825364211507/runs/1fbf4fe7fe314bf489b8ea2f8fee1d6a.
2024/09/29 19:19:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://e2-demo-field-eng.cloud.databricks.com/ml/experiments/4320825364211507.


request_id request response expected_retrieved_context expected_response retrieved_context grading_notes trace response/overall_assessment/rating response/llm_judged/safety/rating response/llm_judged/safety/rationale response/llm_judged/correctness/rating response/llm_judged/correctness/rationale agent/total_input_token_count agent/total_output_token_count agent/total_token_count agent/latency_seconds 397041c2813fbe325566ad2ec6d302e12cd898485f329b406bfc142597b1ffb3 I am reviewing my investment portfolio. What should I do with ICICI stock? I am reviewing my investment portfolio. What should I do with ICICI stock?

 {
 "id": "call_67335176-94df-4556-bfd0-e35cd34abbfb",
 "name": "sarbanimaiti_catalog__agent_demo__finance_stock_quote",
 "arguments": "{\"query\": \"ICICIBANK.NS\"}"
} 

 {
 "id": "call_67335176-94df-4556-bfd0-e35cd34abbfb",
 "content": "{\"format\": \"SCALAR\", \"value\": \"{'quoteResponse': {'result': [{'language': 'en-US', 'region': 'US', 'quoteType': 'EQUITY', 'typeDisp': 'Equity', 'quoteSourceName': 'Delayed Quote', 'triggerable': True, 'customPriceAlertConfidence': 'HIGH', 'currency': 'INR', 'hasPrePostMarketData': False, 'firstTradeDateMilliseconds': 1025495100000, 'priceHint': 2, 'regularMarketChange': -22.550049, 'regularMarketTime': 1727431199, 'regularMarketDayHigh': 1333.0, 'regularMarketDayRange': '1303.85 - 1333.0', 'regularMarketDayLow': 1303.85, 'regularMarketVolume': 17117379, 'regularMarketPreviousClose': 1329.05, 'bid': 0.0, 'ask': 0.0, 'bidSize': 0, 'askSize': 0, 'fullExchangeName': 'NSE', 'financialCurrency': 'INR', 'regularMarketOpen': 1333.0, 'averageDailyVolume3Month': 14346230, 'averageDailyVolume10Day': 18159580, 'fiftyTwoWeekLowChange': 407.5, 'fiftyTwoWeekLowChangePercent': 0.45328143, 'fiftyTwoWeekRange': '899.0 - 1362.35', 'fiftyTwoWeekHighChange': -55.849976, 'fiftyTwoWeekHighChangePercent': -0.040995322, 'fiftyTwoWeekLow': 899.0, 'fiftyTwoWeekHigh': 1362.35, 'fiftyTwoWeekChangePercent': 37.26232, 'earningsTimestamp': 1722077940, 'earningsTimestampStart': 1729335540, 'earningsTimestampEnd': 1729684800, 'earningsCallTimestampStart': 1722081600, 'earningsCallTimestampEnd': 1722081600, 'isEarningsDateEstimate': True, 'trailingAnnualDividendRate': 0.0, 'trailingPE': 20.617012, 'dividendRate': 10.0, 'trailingAnnualDividendYield': 0.0, 'dividendYield': 0.77, 'epsTrailingTwelveMonths': 63.37, 'epsForward': 37.94, 'epsCurrentYear': 30.76, 'priceEpsCurrentYear': 42.47399, 'sharesOutstanding': 7045450240, 'bookValue': 387.523, 'fiftyDayAverage': 1225.958, 'fiftyDayAverageChange': 80.54199, 'fiftyDayAverageChangePercent': 0.065697186, 'twoHundredDayAverage': 1126.1475, 'twoHundredDayAverageChange': 180.35254, 'twoHundredDayAverageChangePercent': 0.16015002, 'marketCap': 9204880375808, 'forwardPE': 34.43595, 'priceToBook': 3.3714128, 'sourceInterval': 15, 'exchange': 'NSI', 'shortName': 'ICICI BANK LTD.', 'longName': 'ICICI Bank Limited', 'messageBoardId': 'finmb_411168', 'exchangeTimezoneName': 'Asia/Kolkata', 'exchangeTimezoneShortName': 'IST', 'gmtOffSetMilliseconds': 19800000, 'market': 'in_market', 'esgPopulated': False, 'regularMarketChangePercent': -1.6967044, 'regularMarketPrice': 1306.5, 'marketState': 'PREPRE', 'exchangeDataDelayedBy': 15, 'averageAnalystRating': '1.4 - Strong Buy', 'tradeable': False, 'cryptoTradeable': False, 'symbol': 'ICICIBANK.NS'}], 'error': None}}\", \"truncated\": false}"
} 

 {
 "id": "call_e644c419-3590-43d8-904e-df7147a1430b",
 "name": "sarbanimaiti_catalog__agent_demo__finance_insight_api",
 "arguments": "{\"query\": \"ICICIBANK.NS\"}"
} 

 {
 "id": "call_e644c419-3590-43d8-904e-df7147a1430b",
 "content": "{\"format\": \"SCALAR\", \"value\": \"[]\", \"truncated\": false}"
} 

**Hold**

Based on the current stock price and financial summary, it is recommended to hold ICICI Bank stock. The current stock price is INR 1306.5, and the 52-week high is INR 1362.35, indicating a slight decrease in value. However, the 52-week low is INR 899.0, showing a significant inc

## Register the model to Unity Catalog

Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "sarbanimaiti_catalog"
schema = "agent_demo"
model_name = "investment_assistant_catalyst"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME)

Successfully registered model 'sarbanimaiti_catalog.agent_demo.investment_assistant_catalyst'.
Created version '1' of model 'sarbanimaiti_catalog.agent_demo.investment_assistant_catalyst'.


## Deploy the agent

In [0]:
from databricks import agents

# Deploy the model to the review app and a model serving endpoint
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version)

To provide a recommendation on whether Hindalco shares are a good choice to invest in, I need to first identify the company's stock ticker symbol. The stock ticker symbol for Hindalco Industries Limited is HINDALCO.

Next, I will use the finance_stock_quote tool to get the current stock price of Hindalco.

<function=sarbanimaiti_catalog__agent_demo__finance_stock_quote>{"query": "HINDALCO"}</function>

Please provide the output of the above function call, and I will proceed with the analysis.

Once I have the current stock price, I will use the finance_insight_api tool to get a summary of the stock's performance.

<function=sarbanimaiti_catalog__agent_demo__finance_insight_api>{"query": "HINDALCO"}</function>

Please provide the output of the above function call, and I will provide a final recommendation on whether to buy, hold, or sell Hindalco shares.

 ------------------------------------------------------------


    Deployment of sarbanimaiti_catalog.agent_demo.investment_assistan

Deployment(model_name='sarbanimaiti_catalog.agent_demo.investment_assistant_catalyst', model_version='1', endpoint_name='agents_sarbanimaiti_catalog-agent_demo-investment_assistant_cat', served_entity_name='sarbanimaiti_catalog-agent_demo-investment_assistant_catalyst_1', query_endpoint='https://e2-demo-field-eng.cloud.databricks.com/serving-endpoints/agents_sarbanimaiti_catalog-agent_demo-investment_assistant_cat/served-models/sarbanimaiti_catalog-agent_demo-investment_assistant_catalyst_1/invocations', endpoint_url='https://e2-demo-field-eng.cloud.databricks.com/ml/endpoints/agents_sarbanimaiti_catalog-agent_demo-investment_assistant_cat', review_app_url='https://e2-demo-field-eng.cloud.databricks.com/ml/review/sarbanimaiti_catalog.agent_demo.investment_assistant_catalyst/1?o=1444828305810485')